In [1]:
# 导入NCS—SK—API
from mvnc import mvncapi as mvnc
# 导入facenet模块包
import facenet_ncs_FPGA
import cv2
import pynq.pynq_computervision.overlays.computer_vision.xf_resize_accel as xv2
import sys
import os
import numpy as np
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
from time import sleep
import realtime_input

# 加载.bit文件
base = BaseOverlay("base.bit")

In [2]:
# 设置摄像头输入分辨率
frame_in_w = 640
frame_in_h = 480
# 打开摄像头
# Configure webcam - note that output images will be BGR
#videoIn = cv2.VideoCapture('/home/xilinx/jupyter_notebooks/ncs-facenet_tensorflow/images/2.jpg')
#videoIn = cv2.VideoCapture('C:\\Users\\Administrator\\face_detect\\face_detect.mp4')
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);

print("Capture device is open: " + str(videoIn.isOpened()))

Capture device is open: True


In [3]:
# 设置HDMI视频输出
hdmi_out = base.video.hdmi_out

# 输出分辨率、通道顺序（与输入一致）
mode = VideoMode(frame_in_w,frame_in_h,24)
hdmi_out.configure(mode, PIXEL_BGR)

# 开启HDMI输出
hdmi_out.start()

In [4]:
# ！！！facenet模型文件路径 ——— facenet.graph ！！！
GRAPH_FILENAME = "/home/xilinx/jupyter_notebooks/ncs-facenet_tensorflow/models/facenet.graph"
# 人脸录入路径
targets_list_dir = '/home/xilinx/jupyter_notebooks/ncs-facenet_tensorflow/targets'

In [5]:
# 检查NCS
devices = mvnc.EnumerateDevices()
if len(devices) == 0:
    print('No NCS devices found')
    quit()
# 开启NCS
device = mvnc.Device(devices[0])
device.OpenDevice()
# 准备模型文件(加载至片上内存)——facenet.graph
graph_file_name = GRAPH_FILENAME
with open(graph_file_name, mode='rb') as f:
    graph_in_memory = f.read()
graph = device.AllocateGraph(graph_in_memory)
# 获取人脸目标路径
temp_list = {}
targets_list = os.listdir(targets_list_dir)
targets_list = [i for i in targets_list if i.endswith('.jpg')]
# ！！！人脸录入 ！！！（获取目标人脸特征集）
targets_feature = facenet_ncs_FPGA.feature(targets_list, temp_list, graph)
print('!!!ok ok ok!!!')

!!!ok ok ok!!!


In [ ]:
base = BareHDMIOverlay('/home/xilinx/jupyter_notebooks/CV/pynq_computervision/overlays/computer_vision/resize.bit')

    
# 实时循环识别人脸
Delay = 0.5
i = 0
while True:
    # 读入一帧图像
    ret, frame1 = videoIn.read()
    #cv2.rectangle(frame, (0+offset, 0+offset),(input_frame.shape[1]-offset-1, input_frame.shape[0]-offset-1),rectangle_color, 10)
    cv2.rectangle(frame1, (200, 80),(440, 400),(0,255,0), 3)
    
    if (base.buttons[0].read()==1):
        sleep(Delay)
        if (base.buttons[0].read()==1):
            for led in base.leds:
                led.toggle()
            targets_feature, targets_list = realtime_input.realtime_input(frame1, hdmi_out, graph, videoIn)
   
    i = i + 1
    # 每隔2帧处理1帧图像
    if(i > 2):
        i = 0
        # ！！！人脸识别 ！！！
        frame2 = frame1[80:400, 200:440]
        frame_res = facenet_ncs_FPGA.run_images(targets_feature, targets_list, graph, frame1, frame2)   
        
        # !!!FPGA—resize!!!
        xv2.resize(inframe, dst=outframe)# 输出结果大小调整
        
        frame_out = hdmi_out.newframe()
          # ！图像由ARM——>FPGA！
        frame_out[:,:,:] = outframe[:,:,:]
        hdmi_out.writeframe(frame_out)